# Morningstar Pro - Entraînement sur Colab
## Système complet de trading algorithmique

### 1. Installation des dépendances

In [ ]:
!pip install -q tensorflow==2.12.0 pandas==1.5.3 numpy==1.23.5 ccxt==4.1.91 ta-lib pyarrow
!pip install -q git+https://github.com/Cabrel10/eva001.git
%load_ext tensorboard

### 2. Configuration initiale

In [ ]:
import sys
sys.path.insert(0, '/content/eva001')
from Morningstar.utils.data_manager import ExchangeDataManager
from Morningstar.workflows.training_workflow import TrainingWorkflow
import pandas as pd
import tensorflow as tf

### 3. Téléchargement des données

In [ ]:
def fetch_data(pairs, timeframe='1d', limit=2000):
    exchange = ExchangeDataManager(exchange="binance")
    all_data = []
    for pair in pairs:
        print(f"Téléchargement {pair}...")
        data = exchange.download_historical_data(pair, timeframe, limit)
        data['pair'] = pair
        all_data.append(data)
    return pd.concat(all_data)

pairs = ["BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT"]
raw_data = fetch_data(pairs)
print(f"Données téléchargées: {raw_data.shape}")

### 4. Préparation du dataset

In [ ]:
from Morningstar.utils.custom_indicators import add_technical_indicators

data = add_technical_indicators(raw_data)
data.to_parquet('full_dataset.parquet')
print(f"Dataset final: {data.shape}")
print(data.head())

### 5. Configuration de l'entraînement

In [ ]:
class ColabConfig:
    def __init__(self):
        self.time_window = 50
        self.features = data.columns.tolist()
        self.epochs = 200
        self.batch_size = 1024
        self.dataset_path = 'full_dataset.parquet'

colab_config = ColabConfig()
workflow = TrainingWorkflow(colab_config)
tf_dataset = workflow._prepare_dataset(data)

### 6. Construction et entraînement du modèle

In [ ]:
with tf.distribute.MirroredStrategy().scope():
    inputs = tf.keras.Input(shape=(50, len(data.columns)))
    x = tf.keras.layers.Conv1D(128, 5, activation='swish')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(256, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(128)(x)
    x = tf.keras.layers.Dense(64, activation='swish')(x)
    outputs = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='huber',
        metrics=['mae']
    )

history = model.fit(
    tf_dataset,
    epochs=colab_config.epochs,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True),
        tf.keras.callbacks.TensorBoard(log_dir='./logs')
    ]
)

### 7. Sauvegarde finale

In [ ]:
model.save('morningstar_pro.h5')
from google.colab import drive
drive.mount('/content/drive')
!cp morningstar_pro.h5 '/content/drive/MyDrive/Colab Data/'
print("Entraînement terminé et modèle sauvegardé")